In [2038]:
import pandas as pd
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import numpy as np
import seaborn as sns
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import re

# Load and Combine DA Data Files

In [2039]:
# Load DA reports
DA2015 = pd.read_csv('DA2015.csv')
DA2016 = pd.read_csv('DA2016.csv')
DA2017 = pd.read_csv('DA2017.csv')


In [2040]:
DA_Full = pd.concat([DA2015,DA2016,DA2017], axis = 0).reset_index(drop = True)
DA_Full.shape

(1185, 290)

# Apply Corrections to Incident Numbers

In [2041]:
# Correct Incident Numbers

DA_Full.IncidentNumber = DA_Full.IncidentNumber.replace( ['2015-15790',
'2015-171',
'2015-18657',
'2015-10854',
'2015-17442',
'2015-18625',
'2015-1903',
'2015-25424',
'2015-382',
'2015-23119',
'2015-24081',
'2015-20102',
'2015-25134',
'2015-25182',
'2015-857',
'2015-7982',
'2016-13631',
'2016-5611',
'2016-2940',
'2016-2840',
'2016-3611',
'2016-5548',
'2016-4220',
'2016-4579',
'2016-6031',
'2016-607',
'2016-6887',
'2016-7005',
'2016-9116',
'2016-92',
'2016-9441',
'2016-16113',
'2016-1073',
'2016-19740',
'2016-197708',
'2016-182477',
'2016-1945'] ,['2015-15970',
'2015-16171',
'2015-16657',
'2015-16854',
'2015-17142',
'2015-18265',
'2015-19035',
'2015-22454',
'2015-23083',
'2015-23719',
'2015-24041',
'2015-25702',
'2015-25734',
'2015-25782',
'2015-851',
'2015-9982',
'2016-1363',
'2016-1564',
'2016-1940',
'2016-2848',
'2016-3414',
'2016-3548',
'2016-4270',
'2016-4578',
'2016-6037',
'2016-6845',
'2016-6882',
'2016-7006',
'2016-9111',
'2016-9219',
'2016-9991',
'2016-10113',
'2016-10735',
'2016-10740',
'2016-17770',
'2016-18247',
'2016-19957'])


In [2042]:
# create combined DF
DA_Full.to_csv('DA_Full.csv')

In [2043]:
DA_Full.columns.values

array(['DIRVersion', 'CaseYear', 'CaseName', 'SuspectActionsImpaired',
       'SuspectActionsDrugOrAlcolol', 'SuspectActionsPushing',
       'SuspectActionsThrewItems', 'SuspectActionsBiting',
       'SuspectActionsInjureChild', 'SuspectActionsSexAssault',
       'SuspectActionsUnwantedContact', 'SuspectActionsDestroyedProp',
       'InjureOtherPerson', 'SuspectActionsShooting',
       'SuspectActionsVerbalAbuse', 'InjurePet', 'SuspectActionsSlapping',
       'SuspectActionsViolatedConditions', 'SuspectActionsForcedEntry',
       'InterferencePhone', 'SuspectActionsSlammingBody',
       'SuspectActionsForciblRestraint', 'SuspectActionsIntimidation',
       'SuspectActionsStabbing', 'SuspectActionsHairPulling',
       'SuspectActionsKicking', 'SuspectActionsStrangulation',
       'SuspectActionsHomicide', 'SuspectActionsPunching',
       'SuspectActionsSuicide', 'SuspectActionsThreats',
       'SuspectActionsInjureKillPerson', 'SuspectActionsInjureKillSelf',
       'SuspectActionsInjure

# Feature Engineerings

In [2044]:
DA_Full.head()

,DIRVersion,CaseYear,CaseName,SuspectActionsImpaired,SuspectActionsDrugOrAlcolol,SuspectActionsPushing,SuspectActionsThrewItems,SuspectActionsBiting,SuspectActionsInjureChild,SuspectActionsSexAssault,...,VictimPainDesc,StatementCounty,PriorCapableKill,PriorKillYouOrChildren,PriorJealous,PriorStrangle,PriorBeatenPregnant,PriorViolenceIncreased,PriorBeliveAbuse,SuspectActionsViolatedCondition
0,1,2015,"(s)Santiago, Krystal (v) Decker, Scott",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,2015,"(s)Siba, Safwan (v) Siba, Sarah",0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1,2015,"(s)Jernigan, John (v) Robinson , Melissa",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1,2015,"(s)Blagbrough, Christopher (v) Coddington, Va...",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1,2015,"(s)Yonetti, Christopher (v) Velez, Mariah",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


# Define function to get suspect and victim name

In [2045]:
def get_suspect(case_name):
    suspect = re.sub('\(s\)','',re.sub(' \(v\)','',re.findall('^\(s\).*\(v\)', case_name)[0]))
    return suspect

In [2046]:
def get_victim(case_name):
    victim = re.sub('\(v\) ','',re.findall('\(v\).*', case_name)[0])
    return victim

In [2047]:
DA_Full['suspect_name'] = DA_Full.CaseName.apply(lambda x: get_suspect(x))
DA_Full['victim_name'] = DA_Full.CaseName.apply(lambda x: get_victim(x))

In [2048]:
DA_Full.tail()

,DIRVersion,CaseYear,CaseName,SuspectActionsImpaired,SuspectActionsDrugOrAlcolol,SuspectActionsPushing,SuspectActionsThrewItems,SuspectActionsBiting,SuspectActionsInjureChild,SuspectActionsSexAssault,...,PriorCapableKill,PriorKillYouOrChildren,PriorJealous,PriorStrangle,PriorBeatenPregnant,PriorViolenceIncreased,PriorBeliveAbuse,SuspectActionsViolatedCondition,suspect_name,victim_name
1180,2,2017,"(s)Smellie, Warren (v) Ector, Frances",0,No,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,No,0,"Smellie, Warren","Ector, Frances"
1181,2,2017,"(s)Adams, Timothy (v) Adams, Sabrina",0,Yes,0,0,0,0,0,...,No,No,No,No,Yes,No,No,0,"Adams, Timothy","Adams, Sabrina"
1182,2,2017,"(s)Dart, William (v) Dart, Sandra",0,NaN,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,No,0,"Dart, William","Dart, Sandra"
1183,2,2017,"(s)Williams, Corina (v) Mickens, Leith",0,Yes,0,0,0,0,0,...,Yes,Yes,Yes,No,No,Yes,NaN,0,"Williams, Corina","Mickens, Leith"
1184,2,2017,"(s)Johnson, Charles (v) Dasilva, Kerri",0,Yes,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,"Johnson, Charles","Dasilva, Kerri"


# Converting Extra columns to binary and replacing NAs with zero

In [2049]:
# Converting no/yes to 0/1
DA_Full.PartiesLiveTogether = DA_Full.PartiesLiveTogether.replace(['No','Yes'], [0,1])
DA_Full.PartiesEverLiveTogether = DA_Full.PartiesEverLiveTogether.replace(['No','Yes'], [0,1])
DA_Full.HaveChildTogher = DA_Full.HaveChildTogher.replace(['No','Yes'], [0,1])

# replacing NAs with zero
DA_Full.PartiesLiveTogether = DA_Full.PartiesLiveTogether.fillna(0)
DA_Full.PartiesEverLiveTogether = DA_Full.PartiesEverLiveTogether.fillna(0)
DA_Full.HaveChildTogher = DA_Full.HaveChildTogher.fillna(0)
DA_Full.PartiesRelationship = DA_Full.PartiesRelationship.fillna('Other')

In [2050]:
#DA_Full.HaveChildTogher.unique()

In [2051]:
#DA_Full.PartiesRelationship.unique()

# Define function to create new column tagging Intimacy

In [2052]:
def is_intimate(df):
    if (df['PartiesRelationship'] == 'Intimate') or (df['PartiesRelationship'] == 'Married') or (df['PartiesRelationship'] == 'FormallyIntimate') or (df['PartiesRelationship'] == 'FormallyMarried'):
        return str('Intimate')
    elif (df['PartiesRelationship'] == 'Other'): # if it is nan or other then do second check
        if df['HaveChildTogher'] == 1:
            return str('Intimate')
        else:
            return str('Not Intimate')
    else:
        return str('Not Intimate')

In [2053]:
DA_Full['Intimacy'] = DA_Full.apply(lambda x: is_intimate(x), axis = 1)

In [2054]:
DA_Full.Intimacy.tail(20)

1165        Intimate
1166        Intimate
1167        Intimate
1168    Not Intimate
1169        Intimate
1170        Intimate
1171        Intimate
1172        Intimate
1173    Not Intimate
1174        Intimate
1175    Not Intimate
1176    Not Intimate
1177        Intimate
1178        Intimate
1179    Not Intimate
1180        Intimate
1181        Intimate
1182        Intimate
1183        Intimate
1184        Intimate
Name: Intimacy, dtype: object

# There are ones NOT labeled Intimate that WE labeled as 'Intimate'

In [2055]:
DA_Full.loc[(DA_Full.PartiesRelationship == 'Other') & (DA_Full.Intimacy == 'Intimate'),['PartiesRelationship','Intimacy']]

,PartiesRelationship,Intimacy
56,Other,Intimate
78,Other,Intimate
127,Other,Intimate
289,Other,Intimate
370,Other,Intimate
443,Other,Intimate
447,Other,Intimate
452,Other,Intimate
561,Other,Intimate
564,Other,Intimate


# Combining Narrative Column from version 1 and version 2

In [2056]:
def get_narrative(df):
    if df.DIRVersion == 1:
        return str(df.ResultsOfInvestigation)
    else:
        return str(df.IncidentComment)

In [2057]:
DA_Full['Narrative'] = DA_Full.apply(lambda x: get_narrative(x), axis = 1)

In [2058]:
DA_Full.head()

,DIRVersion,CaseYear,CaseName,SuspectActionsImpaired,SuspectActionsDrugOrAlcolol,SuspectActionsPushing,SuspectActionsThrewItems,SuspectActionsBiting,SuspectActionsInjureChild,SuspectActionsSexAssault,...,PriorJealous,PriorStrangle,PriorBeatenPregnant,PriorViolenceIncreased,PriorBeliveAbuse,SuspectActionsViolatedCondition,suspect_name,victim_name,Intimacy,Narrative
0,1,2015,"(s)Santiago, Krystal (v) Decker, Scott",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Santiago, Krystal","Decker, Scott",Intimate,P1 states P2 struck/punched him in the face. P...
1,1,2015,"(s)Siba, Safwan (v) Siba, Sarah",0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Siba, Safwan","Siba, Sarah",Not Intimate,See Sup information
2,1,2015,"(s)Jernigan, John (v) Robinson , Melissa",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Jernigan, John","Robinson , Melissa",Intimate,P1 states her and P2 currently live together a...
3,1,2015,"(s)Blagbrough, Christopher (v) Coddington, Va...",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Blagbrough, Christopher","Coddington, Vanessa",Intimate,P1 and P2 got into a verbal argument about som...
4,1,2015,"(s)Yonetti, Christopher (v) Velez, Mariah",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Yonetti, Christopher","Velez, Mariah",Intimate,P1 came into KPD to report her receiving a tex...


In [2059]:
DA_Full.groupby('Intimacy').agg({'Narrative': 'count'})

,Narrative
Intimacy,
Intimate,845
Not Intimate,340


# 71% of records are Intimate

In [2060]:
845/(845+340)

0.7130801687763713

# Tokenize Narrative

In [2061]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('[A-Za-z]+')

DA_Full.Narrative = DA_Full.Narrative.apply(lambda x: tokenizer.tokenize(x))

In [2062]:
DA_Full.head()

,DIRVersion,CaseYear,CaseName,SuspectActionsImpaired,SuspectActionsDrugOrAlcolol,SuspectActionsPushing,SuspectActionsThrewItems,SuspectActionsBiting,SuspectActionsInjureChild,SuspectActionsSexAssault,...,PriorJealous,PriorStrangle,PriorBeatenPregnant,PriorViolenceIncreased,PriorBeliveAbuse,SuspectActionsViolatedCondition,suspect_name,victim_name,Intimacy,Narrative
0,1,2015,"(s)Santiago, Krystal (v) Decker, Scott",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Santiago, Krystal","Decker, Scott",Intimate,"[P, states, P, struck, punched, him, in, the, ..."
1,1,2015,"(s)Siba, Safwan (v) Siba, Sarah",0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Siba, Safwan","Siba, Sarah",Not Intimate,"[See, Sup, information]"
2,1,2015,"(s)Jernigan, John (v) Robinson , Melissa",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Jernigan, John","Robinson , Melissa",Intimate,"[P, states, her, and, P, currently, live, toge..."
3,1,2015,"(s)Blagbrough, Christopher (v) Coddington, Va...",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Blagbrough, Christopher","Coddington, Vanessa",Intimate,"[P, and, P, got, into, a, verbal, argument, ab..."
4,1,2015,"(s)Yonetti, Christopher (v) Velez, Mariah",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Yonetti, Christopher","Velez, Mariah",Intimate,"[P, came, into, KPD, to, report, her, receivin..."


# Remove Stopwords

In [2063]:
from nltk.corpus import stopwords

# adding my own stopwords
stopWords = set(stopwords.words('english'))
stopWords.update(('p1','p2','nan', 'p','see','sup','supp','refer','report', 'party','states', 'state','stated','said','got','told'))

def rem_stop (df):
    return [word.lower() for word in df.Narrative if word.lower() not in stopWords]

In [2064]:
DA_Full.Narrative = DA_Full.apply(lambda x: rem_stop(x), axis = 1)
DA_Full.head()

,DIRVersion,CaseYear,CaseName,SuspectActionsImpaired,SuspectActionsDrugOrAlcolol,SuspectActionsPushing,SuspectActionsThrewItems,SuspectActionsBiting,SuspectActionsInjureChild,SuspectActionsSexAssault,...,PriorJealous,PriorStrangle,PriorBeatenPregnant,PriorViolenceIncreased,PriorBeliveAbuse,SuspectActionsViolatedCondition,suspect_name,victim_name,Intimacy,Narrative
0,1,2015,"(s)Santiago, Krystal (v) Decker, Scott",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Santiago, Krystal","Decker, Scott",Intimate,"[struck, punched, face, showed, obvious, injur..."
1,1,2015,"(s)Siba, Safwan (v) Siba, Sarah",0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Siba, Safwan","Siba, Sarah",Not Intimate,[information]
2,1,2015,"(s)Jernigan, John (v) Robinson , Melissa",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Jernigan, John","Robinson , Melissa",Intimate,"[currently, live, together, residence, leave, ..."
3,1,2015,"(s)Blagbrough, Christopher (v) Coddington, Va...",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Blagbrough, Christopher","Coddington, Vanessa",Intimate,"[verbal, argument, problems, mad, began, trash..."
4,1,2015,"(s)Yonetti, Christopher (v) Velez, Mariah",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Yonetti, Christopher","Velez, Mariah",Intimate,"[came, kpd, receiving, text, facebook, message..."


# Apply Lemmatization


In [2065]:
from nltk import WordNetLemmatizer
lemztr = WordNetLemmatizer()

def porter_stem(df):
    return [lemztr.lemmatize(word) for word in df.Narrative]

In [2066]:
DA_Full.Narrative = DA_Full.apply(lambda x: porter_stem(x), axis = 1)

In [2067]:
DA_Full.head()

,DIRVersion,CaseYear,CaseName,SuspectActionsImpaired,SuspectActionsDrugOrAlcolol,SuspectActionsPushing,SuspectActionsThrewItems,SuspectActionsBiting,SuspectActionsInjureChild,SuspectActionsSexAssault,...,PriorJealous,PriorStrangle,PriorBeatenPregnant,PriorViolenceIncreased,PriorBeliveAbuse,SuspectActionsViolatedCondition,suspect_name,victim_name,Intimacy,Narrative
0,1,2015,"(s)Santiago, Krystal (v) Decker, Scott",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Santiago, Krystal","Decker, Scott",Intimate,"[struck, punched, face, showed, obvious, injur..."
1,1,2015,"(s)Siba, Safwan (v) Siba, Sarah",0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Siba, Safwan","Siba, Sarah",Not Intimate,[information]
2,1,2015,"(s)Jernigan, John (v) Robinson , Melissa",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Jernigan, John","Robinson , Melissa",Intimate,"[currently, live, together, residence, leave, ..."
3,1,2015,"(s)Blagbrough, Christopher (v) Coddington, Va...",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Blagbrough, Christopher","Coddington, Vanessa",Intimate,"[verbal, argument, problem, mad, began, trashi..."
4,1,2015,"(s)Yonetti, Christopher (v) Velez, Mariah",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,"Yonetti, Christopher","Velez, Mariah",Intimate,"[came, kpd, receiving, text, facebook, message..."


In [2068]:
narrative = DA_Full.Narrative

In [2069]:
narrative.head()

0    [struck, punched, face, showed, obvious, injur...
1                                        [information]
2    [currently, live, together, residence, leave, ...
3    [verbal, argument, problem, mad, began, trashi...
4    [came, kpd, receiving, text, facebook, message...
Name: Narrative, dtype: object

# Making the narratives a transaction object

In [2070]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_narrative = te.fit(narrative).transform(narrative)

In [2071]:
te_narrative

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [2072]:
tx_narrative = pd.DataFrame(te_narrative, columns = te.columns_)

In [2073]:
DA_Full.Intimacy.unique()

array(['Intimate', 'Not Intimate'], dtype=object)

# Reduce Dimensionality? Original Total 3805 features

In [2074]:
# Setting minimum amount of times a word must appear to be a feature
min_freq = 2

In [2075]:
# There are 1944 features that appear more than once!
tx_narrative.loc[:,~(tx_narrative.sum() >= min_freq)].head()

,aadvisedof,aaron,aat,abby,ability,abouts,accidentally,account,accuse,acquire,...,yellow,yet,yielded,yoked,zachary,zacharys,zell,zeno,ziane,zoe
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [2076]:
# There are 1861 features that only appear once!
tx_narrative.loc[:,(tx_narrative.sum() >= min_freq)].head()

,abdomen,abeel,able,abortion,abrasion,abuse,abused,abusing,abusive,access,...,yelling,yesterday,ymca,yo,york,young,younger,youngest,yr,zambrella
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [2077]:
tx_narrative = tx_narrative.loc[:, tx_narrative.sum() >= min_freq]
tx_narrative.shape

(1185, 1861)

# Train Test Split

In [2078]:
from sklearn.model_selection import train_test_split

In [2079]:
x = np.array(tx_narrative)
y = np.ravel(DA_Full.Intimacy)

In [2080]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .10, random_state = 0, stratify = y)

# Creating Multinomial Naive Bayes classifier and  Optimizing alpha with gridsearch

In [2081]:
from sklearn import naive_bayes
from sklearn.model_selection import GridSearchCV

mnb_cv = naive_bayes.MultinomialNB()

mnb_param_grid = [{
    'alpha' : np.linspace(.01,5,100) # np.linspace(.01,5,100)
}]

mnb_grid_search = GridSearchCV(mnb_cv,
                              param_grid = mnb_param_grid,
                              cv = 5,
                              scoring = 'accuracy')

In [2082]:
mnb_grid_search.fit(x_train, y_train)
best_alpha = mnb_grid_search.best_params_.get('alpha')
print(best_alpha)
print(mnb_grid_search.best_score_)


2.53020202020202
0.7401500938086304


# Refit Naive Bayes With Best alpha

In [2083]:
mnb = naive_bayes.MultinomialNB(alpha = best_alpha)
mnb.fit(x_train, y_train)

MultinomialNB(alpha=2.53020202020202, class_prior=None, fit_prior=True)

In [2084]:
# Test accuracy : 78.99% (10% split and feature reduction) min freq 2
# Test accuracy : 80.67% (10% split and feature reduction) min freq 1
# Test accuracy : 78.15% (10% split and no feature reduction)

print("The naive score of multinomial naive bayes is: %.4f" %mnb.score(x, y))
print("The training score of multinomial naive bayes is: %.4f" %mnb.score(x_train, y_train))
print("The test score of multinomial naive bayes is: %.4f" %mnb.score(x_test, y_test))

The naive score of multinomial naive bayes is: 0.8346
The training score of multinomial naive bayes is: 0.8377
The test score of multinomial naive bayes is: 0.8067


# MODEL EVALUTATION

In [2085]:
import collections
collections.Counter(y_test)

Counter({'Intimate': 85, 'Not Intimate': 34})

In [2089]:
from sklearn.metrics import confusion_matrix
CM = pd.DataFrame(confusion_matrix( y_test, mnb.predict(x_test),
                                   labels = ['Intimate', 'Not Intimate']),
                columns=['Predicted - Intimate','Predicted - Not Intimate'],
                ).rename(index={0:'Actual - Intimate', 1:'Actual - Not Intimate'})
CM

,Predicted - Intimate,Predicted - Not Intimate
Actual - Intimate,82,3
Actual - Not Intimate,20,14


In [2087]:
print('CORRECT: True Intimate:', 82/(82+20))
print('CORRECT: True Not-Intimate:', 14/(3+14))
print('WRONG: False Intimate:', 20/(82+20))
print('WRONG: False Not-Intimate:', 3/(3+14))
print('****************************************************')
print('TOTAL CORRECT: ', (82+14)/(82+14+20+3) )
print('TOTAL WRONG: ', (20+3)/(82+14+20+3) )

CORRECT: True Intimate: 0.803921568627451
CORRECT: True Not-Intimate: 0.8235294117647058
WRONG: False Intimate: 0.19607843137254902
WRONG: False Not-Intimate: 0.17647058823529413
****************************************************
TOTAL CORRECT:  0.8067226890756303
TOTAL WRONG:  0.19327731092436976
